In [226]:
# Creating an options base class

class StockOption(object):
    def __init__(self, S0, K, r = 0.05, T = 1, N = 2, pu = 0, pd = 0, div = 0, sigma = 0, is_put = False, is_am = False):
    # S0 = initial stock price
    # K = strike price
    # r = risk-free rate
    # T = time to maturity
    # N = number of time steps
    # pu = probability at up-state
    # pd = probability at down-state
    # div = dividend yield
    # is_put = true for puts, false for calls
    # is_am = true for American option, false for European
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.N = max(1, N)
        self.pu = pu
        self.pd = pd
        self.div = div
        self.sigma = sigma
        self.is_call = not is_put
        self.is_european = not is_am
        self.STs = [] # Declares the stock prices tree
        
        
        # Find value of a single time step (in years)
    @property
    def dt(self):
        return self.T/float(self.N)
        
    # Find discount factor
    @property
    def df(self):
        return math.exp(-(self.r - self.div)*self.dt)

In [227]:
import numpy as np
import math
from decimal import Decimal

# make a Euro subclass
class BinomialEuropeanOption(StockOption):
    def setup_parameters(self):
        self.M = self.N+1 # number of terminal nodes in tree
        self.u = 1+self.pu
        self.d = 1-self.pd
        # solve for the risk-neutral probability of investing in the stock
        self.qu = (math.exp((self.r-self.div)*self.dt)-self.d) / (self.u-self.d)
        self.qd = 1-self.qu
        
    def init_stock_price_tree(self):
        # initialize terminal price nodes to 0
        self.STs = np.zeros(self.M)
        
        # calculate expected stock prices for each node
        for i in range(self.M):
            self.STs[i] = self.S0*(self.u**(self.N-i)) * (self.d**i)
        # print('Stock tree is: ', self.STs)
    
    def init_payoffs_tree(self):
        # return payoffs when reaching terminal nodes
        if self.is_call:
            return np.maximum(0, self.STs - self.K)
        else:
            return np.maximum(0, self.K - self.STs)
        
    def traverse_tree(self, payoffs):
        # calculate discounted payoffs at each node
        # print('pre-discounted payoffs are', payoffs)
        # print('discount factor is: ', self.df)
        # print('qu is', self.qu)
        # print('qd is', self.qd)
        
        # starts at terminal nodes - discounts one level at a time weighing by risk neutral probabilities
        for i in range(self.N):
            payoffs = (payoffs[:-1]*self.qu + payoffs[1:]*self.qd)*self.df
            # print('payoffs are', payoffs)
        return payoffs
        
    def begin_tree_traversal(self):
        payoffs = self.init_payoffs_tree()
        return self.traverse_tree(payoffs)
    
    def price(self):
        # delivers the price of the option
        self.setup_parameters()
        self.init_stock_price_tree()
        payoffs = self.begin_tree_traversal()
        return payoffs[0]

In [228]:
# run an option
eu_option = BinomialEuropeanOption(50, 52, r=0.05, T=4, N=4, pu=0.2, pd=0.2, is_put=True)
eu_option1 = eu_option.price()
print('Euro put option price is: ', eu_option1) 

Euro put option price is:  4.330196749669495


In [229]:
# make an American subclass
class BinomialTreeOption(StockOption):
    def setup_parameters(self):
        self.u = 1+self.pu
        self.d = 1-self.pd
        # solve for the risk-neutral probability of investing in the stock
        self.qu = (math.exp((self.r-self.div)*self.dt)-self.d) / (self.u-self.d)
        self.qd = 1-self.qu
    
    # use 2d numpy array to store expected returns of the stock prices for all time steps
    def init_stock_price_tree(self):
        # initialize 2D tree at T=0
        # we need to store every node because it could be exercised at any time
        self.STs = [np.array([self.S0])]
        
        # simulate the possible stock prices path, stores every node in that 2d array
        # print('Finding the stock prices path: ')
        for i in range(self.N):
            prev_branches = self.STs[-1]
            # print('prev_branches is: ', prev_branches)
            st = np.concatenate((prev_branches*self.u, [prev_branches[-1]*self.d]))
            # print('this tree is:', st)
            self.STs.append(st) # add nodes at each time step
            # print('Whole tree is:', self.STs)
    
    # Create the payoff tree as 2d numpy array, starting w intrinsic values of the option at maturity
    def init_payoffs_tree(self):
        if self.is_call:
            return np.maximum(0, self.STs[self.N]-self.K)
        else:
            return np.maximum(0, self.K-self.STs[self.N])
    
    # returns max payoff values between exercising American option early and not exercising at all
    def check_early_exercise(self, payoffs, node):
        if self.is_call:
            curr_node_payoff = self.STs[node]-self.K
            return np.maximum(payoffs, curr_node_payoff)
        else:
            curr_node_payoff =  self.K-self.STs[node]
            # print('curr_node_payoff is', curr_node_payoff[1])
            # print('comparing payoffs of:', payoffs, 'and', curr_node_payoff)
            return np.maximum(payoffs, curr_node_payoff)

    def traverse_tree(self, payoffs):
        # starting at the terminal nodes
        # print('payoffs before:', payoffs)
        for i in reversed(range(self.N)):
            # Payoffs from not exercising the option
            payoffs = (payoffs[:-1]*self.qu + payoffs[1:]*self.qd)*self.df
            # print('Payoffs from not exercising the option:', payoffs)

            # payoffs from exercising, for American options
            if not self.is_european:
                payoffs = self.check_early_exercise(payoffs, i)
                # print('payoffs after early check is: ', payoffs)
        return payoffs

    def begin_tree_traversal(self):
        payoffs = self.init_payoffs_tree()
        # print('non-discounted payoffs of terminal nodes: ', payoffs)
        return self.traverse_tree(payoffs)

    def price(self):
        # delivers the price of the option
        self.setup_parameters()
        self.init_stock_price_tree()
        payoffs = self.begin_tree_traversal()
        return payoffs[0]

In [230]:
# test for an American option

am_option = BinomialTreeOption(50, 52, r=0.05, T=2, N=2, pu=0.2, pd=0.2, is_put=True, is_am=True)
print ('American put option price is: ', am_option.price())

American put option price is:  5.089632474198373


In [231]:
# Cox-Rubenstein model proposes binomial model matching the mean and variance of asset
class BinomialCRROption(BinomialTreeOption):
    def setup_parameters(self):
        self.u = math.exp(self.sigma*math.sqrt(self.dt))
        self.d = 1/self.u
        self.qu = (math.exp((self.r-self.div)*self.dt)-self.d) / (self.u-self.d)
        self.qd = 1-self.qu

In [232]:
# Run a CRR binomial
eu_option = BinomialCRROption(50, 52, r=0.05, T=2, N=2, sigma=0.3, is_put=True)
eu_option2 = eu_option.price()

eu_option_list = []
eu_option_list.append(eu_option1)
eu_option_list.append(eu_option2)

In [233]:
# Leisen-Reimer tree approximates Black Scholes solution, uses inversion formula to achieve better accuracy
class BinomialLROption(BinomialTreeOption):
    def setup_parameters(self):
        odd_N = self.N if (self.N % 2 == 0) else (self.N+1)
        d1 = (math.log(self.S0/self.K) + ((self.r-self.div) + (self.sigma**2)/2)*self.T) / (self.sigma*math.sqrt(self.T))
        d2 = (math.log(self.S0/self.K) - ((self.r-self.div) + (self.sigma**2)/2)*self.T) / (self.sigma*math.sqrt(self.T))
        pbar = self.pp_2_inversion(d1, odd_N)
        self.p = self.pp_2_inversion(d2, odd_N)
        self.u = 1/self.df * pbar/self.p
        self.d = (1/self.df - self.p*self.u)/(1-self.p)
        self.qu = self.p
        self.qd = 1-self.p
        
    def pp_2_inversion(self, z, n):
        return .5 + math.copysign(1,z)*math.sqrt(.25-.25*math.exp(-((z/(n+1./3.+.1/(n+1)))**2.)*(n+1/6)))

In [234]:
# run a Leisen-Reimer
eu_option = BinomialLROption(50, 52, r=0.05, T=2, N=4, sigma=0.3, is_put=True)
eu_option3 = eu_option.price()
eu_option_list.append(eu_option3)
# PRICED THE LR WRONG FIX LATER

In [235]:
# Because the nodes in CRR Binomial recombine at every other step, we can store unique nodes in
# a lattice to reduce computing power
class BinomialCRRLattice(BinomialCRROption):
    def setup_parameters(self):
        super(BinomialCRRLattice, self).setup_parameters() #inherits setup param from binomialCRR
        self.M = 2*self.N+1
        
    def init_stock_price_tree(self):
        self.STs = np.zeros(self.M)
        self.STs[0] = self.S0*self.u**self.N # 1st index is if goes up at every time step
        
        for i in range(self.M)[1:]:
            self.STs[i] = self.STs[i-1]*self.d # calculate rest of the indices
            
    def init_payoffs_tree(self):
        odd_nodes = self.STs[::2]
        print(odd_nodes)
        if self.is_call:
            return np.maximum(0, odd_nodes-self.K)
        else:
            return np.maximum(0, self.K-odd_nodes)
        
    def check_early_exercise(self, payoffs, node):
        self.STs = self.STs[1:-1]
        odd_STs = self.STs[::2]
        print(odd_STs)
        if self.is_call:
            curr_node_payoff = odd_STs-self.K
            return np.maximum(payoffs, curr_node_payoff)
        else:
            curr_node_payoff =  self.K-odd_STs
            # print('curr_node_payoff is', curr_node_payoff[1])
            # print('comparing payoffs of:', payoffs, 'and', curr_node_payoff)
            return np.maximum(payoffs, curr_node_payoff)

In [236]:
eu_option = BinomialCRRLattice(50, 52, r = 0.05, T = 2, N = 2, sigma = 0.3, is_put = True)
eu_option_list.append(eu_option.price())

j = 1
# print(len(eu_option_list))
for i in eu_option_list:
    print('EU Option', j, 'is priced at', i)
    j+=1

[91.10594002 50.         27.4405818 ]
EU Option 1 is priced at 4.330196749669495
EU Option 2 is priced at 6.245708445206436
EU Option 3 is priced at 14.39994943859485
EU Option 4 is priced at 6.245708445206432
